In [29]:
import pandas as pd
import lightgbm

In [9]:
data1=pd.read_csv('比赛训练集.csv',encoding='gbk')
data2=pd.read_csv('比赛测试集.csv',encoding='gbk')
#label标记为-1
data2['患有糖尿病标识']=-1
#训练集和测试机合并
data=pd.concat([data1,data2],axis=0,ignore_index=True)
data

,编号,性别,出生年份,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,患有糖尿病标识
0,1,0,1996,30.1,无记录,106.0,3.818,7.89,0.0,0
1,2,0,1988,27.5,无记录,84.0,-1.000,0.00,14.7,0
2,3,1,1988,36.5,无记录,85.0,7.131,0.00,40.1,1
3,4,1,1992,29.5,无记录,91.0,7.041,0.00,0.0,0
4,5,0,1998,42.0,叔叔或者姑姑有一方患有糖尿病,NaN,7.134,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
6065,996,1,1990,50.1,无记录,87.0,5.125,0.00,0.0,-1
6066,997,0,1992,56.3,无记录,87.0,7.695,0.00,0.0,-1
6067,998,1,1992,23.8,无记录,85.0,3.194,7.50,0.0,-1
6068,999,0,2000,53.1,无记录,95.0,8.226,7.55,0.0,-1


In [20]:
"""
人体的成人体重指数正常值是在18.5-24之间
低于18.5是体重指数过轻
在24-27之间是体重超重
27以上考虑是肥胖
高于32了就是非常的肥胖。
"""
def BMI(a):
    if a<18.5:
        return 0
    elif 18.5<=a<=24:
        return 1
    elif 24<a<=27:
        return 2
    elif 27<a<=32:
        return 3
    else:
        return 4
    
data['BMI']=data['体重指数'].apply(BMI)
data['出生年份']=2022-data['出生年份']  #换成年龄
#糖尿病家族史
"""
无记录
叔叔或者姑姑有一方患有糖尿病/叔叔或姑姑有一方患有糖尿病
父母有一方患有糖尿病
"""
def FHOD(a):
    if a=='无记录':
        return 0
    elif a=='叔叔或者姑姑有一方患有糖尿病' or a=='叔叔或姑姑有一方患有糖尿病':
        return 1
    else:
        return 2
    
data['糖尿病家族史']=data['糖尿病家族史'].apply(FHOD)
data['舒张压']=data['舒张压'].fillna(-1)
"""
舒张压范围为60-90
"""
def DBP(a):
    if a<60:
        return 0
    elif 60<=a<=90:
        return 1
    elif a>90:
        return 2
    else:
        return a
data['DBP']=data['舒张压'].apply(DBP)
data

,编号,性别,出生年份,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,患有糖尿病标识,BMI,DBP
0,1,0,26,30.1,0,106.0,3.818,7.89,0.0,0,3,2
1,2,0,34,27.5,0,84.0,-1.000,0.00,14.7,0,3,1
2,3,1,34,36.5,0,85.0,7.131,0.00,40.1,1,4,1
3,4,1,30,29.5,0,91.0,7.041,0.00,0.0,0,3,2
4,5,0,24,42.0,1,-1.0,7.134,0.00,0.0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6065,996,1,32,50.1,0,87.0,5.125,0.00,0.0,-1,4,1
6066,997,0,30,56.3,0,87.0,7.695,0.00,0.0,-1,4,1
6067,998,1,30,23.8,0,85.0,3.194,7.50,0.0,-1,1,1
6068,999,0,22,53.1,0,95.0,8.226,7.55,0.0,-1,4,2


In [51]:
train=data[data['患有糖尿病标识'] !=-1]
test=data[data['患有糖尿病标识'] ==-1]
train_label=train['患有糖尿病标识']
train=train.drop(['编号','患有糖尿病标识'],axis=1)
test=test.drop(['编号','患有糖尿病标识'],axis=1)
test

,性别,出生年份,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,BMI,DBP
5070,0,35,33.1,0,72.0,6.586,24.16,2.94,4,1
5071,0,24,20.6,1,68.0,3.861,0.00,0.00,1,1
5072,1,43,42.1,0,98.0,5.713,0.00,3.53,4,2
5073,0,23,34.6,0,66.0,4.684,0.00,3.14,4,1
5074,0,25,27.7,0,89.0,7.948,14.65,2.65,3,1
...,...,...,...,...,...,...,...,...,...,...
6065,1,32,50.1,0,87.0,5.125,0.00,0.00,4,1
6066,0,30,56.3,0,87.0,7.695,0.00,0.00,4,1
6067,1,30,23.8,0,85.0,3.194,7.50,0.00,1,1
6068,0,22,53.1,0,95.0,8.226,7.55,0.00,4,2


In [68]:
def select_by_lgb(train_data,train_label,test_data,random_state=2022,metric='auc',num_round=300):
    clf=lightgbm
    train_matrix=clf.Dataset(train_data,label=train_label)

    params={
            'boosting_type': 'gbdt',  
            'objective': 'binary',
            'learning_rate': 0.1,
            'metric': metric,
            'seed': 2020,
            'nthread':-1 }
    model=clf.train(params,train_matrix,num_round)
    pre_y=model.predict(test_data)
    return pre_y

test_data=select_by_lgb(train,train_label,test)

[LightGBM] [Info] Number of positive: 1936, number of negative: 3134
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1063
[LightGBM] [Info] Number of data points in the train set: 5070, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.381854 -> initscore=-0.481686
[LightGBM] [Info] Start training from score -0.481686


In [72]:
pre_y=pd.DataFrame(test_data)
pre_y['label']=pre_y[0].apply(lambda x:1 if x>0.5 else 0)
pre_y

,0,label
0,0.041986,0
1,0.000003,0
2,0.000328,0
3,0.002007,0
4,0.094745,0
...,...,...
995,0.000011,0
996,0.999985,1
997,0.000006,0
998,0.999976,1


In [73]:
result=pd.read_csv('提交示例.csv')
result['label']=pre_y['label']
result.to_csv('baseline.csv',index=False)